In [31]:
import pandas as pd
import plotly.graph_objects as go
from  binance.client import  Client
import plotly.express as px
import numpy as np

In [32]:
client=Client('','')
start_time="2024-08-01 00:00:00"
end_time="2024-09-01 00:00:00"
candles=client.get_historical_klines('BTCUSDT',Client.KLINE_INTERVAL_4HOUR,start_time,end_time,1500)
# 转换为DataFrame
df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
# 只保留需要的列
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# 将时间戳转换为可读时间
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['open']=df['open'].astype(float)
df['high']=df['high'].astype(float)
df['low']=df['low'].astype(float)
df['close']=df['close'].astype(float)
df['volume']=df['volume'].astype(float)

In [33]:
# df.to_csv('2408-2409_4h.csv')

In [34]:
# 设置时间索引
df.set_index('timestamp', inplace=True)

# 计算RSI
def calculate_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['EMA_12'] = df['close'].ewm(span=12,adjust=False).mean()
df['EMA_26'] = df['close'].ewm(span=26,adjust=False).mean()
# 判断趋势信号
df['Signal'] = 0
df['Signal'][df['EMA_12'] > df['EMA_26']] = 1
df['Signal'][df['EMA_12'] < df['EMA_26']] = -1

# 查找信号切换点
df['Position'] = df['Signal'].diff()

# 只显示信号切换点：-2表示从买入到卖出，2表示从卖出到买入
buy_signals = df[df['Position'] == 2]
sell_signals = df[df['Position'] == -2]
# 计算每笔交易的收益
profit = 0

df['RSI'] = calculate_rsi(df['close'], 14)
# 确定超买和超卖点
overbought = df[df['RSI'] > 70]
oversold = df[df['RSI'] < 30]
for i in range(min(len(buy_signals), len(sell_signals))):
    buy_price = buy_signals['close'].iloc[i]
    sell_price = sell_signals['close'].iloc[i]
    
    # 多单收益：买入到卖出
    profit += sell_price - buy_price
    
    if i < len(buy_signals) - 1:
        # 空单收益：卖出到下一次买入
        next_buy_price = buy_signals['close'].iloc[i + 1]
        profit += sell_price - next_buy_price

# 输出总收益
print(f"Total profit: {profit} USD")
# df['SMA_200'] = df['close'].rolling(window=200).mean()    
fig=go.Figure(data=go.Candlestick(x=df.index,
                                  open=df['open'],
                                  high=df['high'],
                                  low=df['low'],
                                  close=df['close'],
                                  name='4h Kline'
                                  ))
fig.add_trace(go.Scatter(x=df.index, y=df['close'], mode='lines', name='BTC Price'))
# fig.add_trace(go.Scatter(x=df.index, y=df['EMA_12'], mode='lines', name='EMA 12'))
# fig.add_trace(go.Scatter(x=df.index, y=df['EMA_26'], mode='lines', name='EMA 26'))
# # fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], mode='lines', name='SMA 200'))
# 添加EMA曲线
fig.add_trace(go.Scatter(x=df.index, y=df['EMA_12'], line=dict(color='blue', width=1), name='12-period EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df['EMA_26'], line=dict(color='orange', width=1), name='26-period EMA'))

# 添加买入信号点
fig.add_trace(go.Scatter(x=buy_signals.index,
                         y=df['EMA_12'][df['Signal'] == 1],
                         mode='markers',
                         marker=dict(color='green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

# 添加卖出信号点
fig.add_trace(go.Scatter(x=sell_signals.index,
                         y=df['EMA_12'][df['Signal'] == -1],
                         mode='markers',
                         marker=dict(color='red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

# 添加超买点和超卖点
fig.add_trace(go.Scatter(x=overbought.index,
                         y=overbought['close'],
                         mode='markers',
                         marker=dict(color='purple', size=8, symbol='x'),
                         name='Overbought (RSI > 70)'))

fig.add_trace(go.Scatter(x=oversold.index,
                         y=oversold['close'],
                         mode='markers',
                         marker=dict(color='cyan', size=8, symbol='x'),
                         name='Oversold (RSI < 30)'))
# 添加RSI图表
fig_rsi = go.Figure()

fig_rsi.add_trace(go.Scatter(x=df.index, y=df['RSI'], line=dict(color='black', width=2), name='RSI'))

# 标注超买和超卖区域
fig_rsi.add_shape(type="rect", xref="x", yref="paper", x0=df.index.min(), y0=70, x1=df.index.max(), y1=100,
                  fillcolor="purple", opacity=0.3, layer="below", line_width=0, name='Overbought Zone (70-100)')

fig_rsi.add_shape(type="rect", xref="x", yref="paper", x0=df.index.min(), y0=0, x1=df.index.max(), y1=30,
                  fillcolor="cyan", opacity=0.3, layer="below", line_width=0, name='Oversold Zone (0-30)')
fig_rsi.update_layout(title='RSI Over Time',
                      xaxis_title='Time',
                      yaxis_title='RSI')
# 更新图表布局
fig.update_layout(title='Bitcoin 4H Candlestick Chart with EMA',
                  xaxis_title='Time',
                  yaxis_title='Price (USD)',
                  xaxis_rangeslider_visible=False)

# 显示图表
fig.show()
fig_rsi.show()

Total profit: -6040.149999999987 USD


/tmp/ipykernel_41414/4097445065.py:17: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/tmp/ipykernel_41414/4097445065.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [35]:
# 构建收益表
trades = []
trade_pairs = min(len(buy_signals), len(sell_signals))

for i in range(trade_pairs):
    buy_time = buy_signals.index[i]
    sell_time = sell_signals.index[i]
    buy_price = buy_signals['close'].iloc[i]
    sell_price = sell_signals['close'].iloc[i]
    
    # 多单收益：买入到卖出
    profit = sell_price - buy_price
    
    if i < trade_pairs - 1:
        # 空单收益：卖出到下一次买入
        next_buy_price = buy_signals['close'].iloc[i + 1]
        profit += sell_price - next_buy_price
    
    trades.append({
        'Buy Time': buy_time,
        'Sell Time': sell_time,
        'Buy Price': buy_price,
        'Sell Price': sell_price,
        'Profit': profit
    })

# 创建收益表的DataFrame
profit_df = pd.DataFrame(trades)

# 累积收益
profit_df['Cumulative Profit'] = profit_df['Profit'].cumsum()

# 显示收益表
print(profit_df)

# 使用Plotly绘制累积收益曲线
fig = px.line(profit_df, x='Sell Time', y='Cumulative Profit', title='Cumulative Profit Over Time')
fig.update_layout(xaxis_title='Sell Time', yaxis_title='Cumulative Profit (USD)')
fig.show()

             Buy Time           Sell Time  Buy Price  Sell Price   Profit  \
0 2024-08-01 20:00:00 2024-08-01 12:00:00   65354.02    62906.60 -1226.81   
1 2024-08-08 20:00:00 2024-08-02 16:00:00   61685.99    62633.57  2761.00   
2 2024-08-13 16:00:00 2024-08-13 00:00:00   60820.15    59040.00 -2232.14   
3 2024-08-17 20:00:00 2024-08-15 00:00:00   59491.99    58442.98 -3225.21   
4 2024-08-20 00:00:00 2024-08-19 04:00:00   60619.18    58502.19 -2116.99   

   Cumulative Profit  
0           -1226.81  
1            1534.19  
2            -697.95  
3           -3923.16  
4           -6040.15  


In [37]:

df['RSI'] = calculate_rsi(df['close'])

# 计算短期和长期EMA
df['EMA_short'] = df['close'].ewm(span=12, adjust=False).mean()
df['EMA_long'] = df['close'].ewm(span=26, adjust=False).mean()
# 定义买入和卖出信号
df['Buy'] = np.where((df['RSI'] < 30) & (df['EMA_short'] > df['EMA_long']), df['close'], np.nan)
df['Sell'] = np.where((df['RSI'] > 70) & (df['EMA_short'] < df['EMA_long']), df['close'], np.nan)
# 计算收益
initial_balance = 10000  # 初始资金
balance = initial_balance
position = 0
trades = []
for i in range(1, len(df)):
    if not np.isnan(df['Buy'].iloc[i]) and position == 0:
        # 买入
        position = balance / df['Buy'].iloc[i]
        balance = 0
        trades.append(('Buy', df.index[i], df['Buy'].iloc[i]))
    elif not np.isnan(df['Sell'].iloc[i]) and position > 0:
        # 卖出
        balance = position * df['Sell'].iloc[i]
        position = 0
        trades.append(('Sell', df.index[i], df['Sell'].iloc[i]))

# 如果最后仍然持有仓位，则计算最后的市值
if position > 0:
    balance = position * df['close'].iloc[-1]
    trades.append(('Sell', df.index[-1], df['close'].iloc[-1]))

# 计算收益
total_profit = balance - initial_balance
print(f"Total Profit: {total_profit:.2f} USD")
# 将交易记录转换为DataFrame
trades_df = pd.DataFrame(trades, columns=['Action', 'Date', 'Price'])

# 绘制K线图及交易信号
fig_rsk = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     name='Candlestick')])

fig_rsk.add_trace(go.Scatter(x=trades_df[trades_df['Action'] == 'Buy']['Date'],
                         y=trades_df[trades_df['Action'] == 'Buy']['Price'],
                         mode='markers',
                         marker=dict(color='green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

fig_rsk.add_trace(go.Scatter(x=trades_df[trades_df['Action'] == 'Sell']['Date'],
                         y=trades_df[trades_df['Action'] == 'Sell']['Price'],
                         mode='markers',
                         marker=dict(color='red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

fig.add_trace(go.Scatter(x=df.index, y=df['EMA_short'],
                         mode='lines', line=dict(color='blue', width=1),
                         name='EMA Short'))

fig_rsk.add_trace(go.Scatter(x=df.index, y=df['EMA_long'],
                         mode='lines', line=dict(color='orange', width=1),
                         name='EMA Long'))

fig_rsk.update_layout(title='Bitcoin 4H Candlestick Chart with Combined RSI and EMA Signals',
                  xaxis_title='Time',
                  yaxis_title='Price (USD)',
                  xaxis_rangeslider_visible=False)

fig_rsk.show()

Total Profit: 2.51 USD
